In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,736440.19,-10.66,3.03,-3.09,-1.44,0.15,0.39
4,-10.66,465251.45,-5.19,-1.02,0.40,0.23,-0.05
5,3.03,-5.19,108830.78,-1.70,0.63,0.70,0.04
6,-3.09,-1.02,-1.70,47223.61,0.20,-0.23,0.02
7,-1.44,0.40,0.63,0.20,9148.20,-0.01,0.01
8,0.15,0.23,0.70,-0.23,-0.01,918.80,0.01
9,0.39,-0.05,0.04,0.02,0.01,0.01,78.41


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00002,0.00001,-0.00002,-0.00002,0.00001,0.00005
4,-0.00002,1.00000,-0.00002,-0.00001,0.00001,0.00001,-0.00001
5,0.00001,-0.00002,1.00000,-0.00002,0.00002,0.00007,0.00001
6,-0.00002,-0.00001,-0.00002,1.00000,0.00001,-0.00003,0.00001
7,-0.00002,0.00001,0.00002,0.00001,1.00000,-0.00000,0.00001
8,0.00001,0.00001,0.00007,-0.00003,-0.00000,1.00000,0.00002
9,0.00005,-0.00001,0.00001,0.00001,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

18.261673133746864

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.89103016e+10 2.96305133e+10 1.59785946e+09 2.75892173e+08
 9.27526915e+06 7.06770017e+04 3.26884308e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.985681,-0.166951,-0.021730,-0.009243,-0.001784,-0.000179,-0.000015
1,0.165356,0.984516,-0.053689,-0.022017,-0.004213,-0.000423,-0.000036
2,0.028365,0.046170,0.989600,-0.131765,-0.019750,-0.001960,-0.000168
3,0.016393,0.026066,0.128551,0.988725,-0.070031,-0.006686,-0.000570
4,0.004148,0.006563,0.028179,0.066296,0.996049,-0.051178,-0.004300
5,0.000618,0.000977,0.004174,0.009641,0.049989,0.995040,-0.085355
6,0.000106,0.000168,0.000717,0.001654,0.008538,0.085019,0.996341


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.014319239792624505,
 0.015483749870565111,
 0.010400097740553838,
 0.011275025698940344,
 0.003951390350531603,
 0.004960387215124151,
 0.0036588818843362736]